<a href="https://colab.research.google.com/github/devadharshini2005/intel_ai_tutor/blob/main/intel_unati.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# ✅ Step 1: Generate Synthetic Student Data
np.random.seed(42)
num_samples = 1000

data = {
    "Name": [f"Student_{i}" for i in range(num_samples)],
    "Age": np.random.randint(10, 18, num_samples),
    "Gender": np.random.choice(["Male", "Female"], num_samples),
    "Country": np.random.choice(["India", "USA", "UK"], num_samples),
    "State": np.random.choice(["State1", "State2", "State3"], num_samples),
    "City": np.random.choice(["City1", "City2", "City3"], num_samples),
    "Parent_Occupation": np.random.choice(["Engineer", "Doctor", "Teacher"], num_samples),
    "Earning_Class": np.random.choice(["Low", "Middle", "High"], num_samples),
    "Student_Level": np.random.choice(["Beginner", "Intermediate", "Advanced"], num_samples),
    "Course_Level": np.random.choice(["Basic", "Intermediate", "Advanced"], num_samples),
    "Course_Name": np.random.choice(["Math", "Science", "English"], num_samples),
    "Study_Hours": np.random.uniform(0.5, 6, num_samples),
    "Material_Name": np.random.choice(["Book1", "Book2", "Video1"], num_samples),
    "Material_Level": np.random.choice(["Easy", "Medium", "Hard"], num_samples),
    "IQ_Score": np.random.randint(80, 140, num_samples),
    "Assessment_Score": np.random.randint(40, 100, num_samples),
}

df = pd.DataFrame(data)

# ✅ Step 2: Encode Categorical Features
categorical_cols = ["Gender", "Country", "State", "City", "Parent_Occupation",
                    "Earning_Class", "Student_Level", "Course_Level",
                    "Course_Name", "Material_Name", "Material_Level"]

encoders = {col: LabelEncoder() for col in categorical_cols}

for col in categorical_cols:
    df[col] = encoders[col].fit_transform(df[col])

# ✅ Step 3: Prepare Training Data
X = df.drop(columns=["Name", "Assessment_Score"])  # Exclude Name & Target
y = df["Assessment_Score"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Step 4: Train ML Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# ✅ Step 5: Save Model & Encoders
joblib.dump(model, "pretrained_assessment_model.pkl")
joblib.dump(encoders, "encoders.pkl")
print("✅ Model trained & saved successfully!")


✅ Model trained & saved successfully!


In [ ]:
def predict_assessment(student_data):
    # Load Model & Encoders
    model = joblib.load("pretrained_assessment_model.pkl")
    encoders = joblib.load("encoders.pkl")

    # Convert Input Data into DataFrame
    df = pd.DataFrame([student_data])

    # Encode Categorical Values Safely (Handle Unseen Labels)
    for col in encoders:
        if df[col][0] in encoders[col].classes_:
            df[col] = encoders[col].transform([df[col][0]])  # Convert existing string to numeric
        else:
            df[col] = -1  # Assign new values as -1 (unknown category)

    # Ensure Column Order Matches Training Data
    df = df[X.columns]  # Keep only relevant features

    # Predict Assessment Score
    prediction = model.predict(df)[0]

    return round(prediction, 2)


# ✅ Step 6: Track Student Progress
def update_progress():
    student_data = {
        "Age": int(input("Enter Age: ")),
        "Gender": input("Enter Gender (Male/Female): "),
        "Country": input("Enter Country: "),
        "State": input("Enter State: "),
        "City": input("Enter City: "),
        "Parent_Occupation": input("Enter Parent Occupation: "),
        "Earning_Class": input("Enter Earning Class (Low/Middle/High): "),
        "Student_Level": input("Enter Student Level (Beginner/Intermediate/Advanced): "),
        "Course_Level": input("Enter Course Level (Basic/Intermediate/Advanced): "),
        "Course_Name": input("Enter Course Name (Math/Science/English): "),
        "Study_Hours": float(input("Enter Study Hours per day: ")),
        "Material_Name": input("Enter Material Name: "),
        "Material_Level": input("Enter Material Level (Easy/Medium/Hard): "),
        "IQ_Score": int(input("Enter IQ Score: ")),
    }

    predicted_score = predict_assessment(student_data)

    # Determine Promotion or Demotion
    if predicted_score >= 60:
        print(f"🎉 Student is PROMOTED! Predicted Score: {predicted_score}")
    else:
        print(f"📉 Student is DEMOTED. Predicted Score: {predicted_score}")

update_progress()


Enter Age: 9
Enter Gender (Male/Female): Female
Enter Country: India
Enter State: Tamil Nadu
Enter City: Pudukkottai
Enter Parent Occupation: Employee
Enter Earning Class (Low/Middle/High): Middle
Enter Student Level (Beginner/Intermediate/Advanced): Intermediate
Enter Course Level (Basic/Intermediate/Advanced): Intermediate
Enter Course Name (Math/Science/English): Science
Enter Study Hours per day: 3
Enter Material Name: Book
Enter Material Level (Easy/Medium/Hard): Medium
Enter IQ Score: 90
🎉 Student is PROMOTED! Predicted Score: 66.03


In [ ]:
import pandas as pd
import numpy as np
import joblib

# Load the trained model
model = joblib.load("pretrained_assessment_model.pkl")

# List of features used for training
X = df.drop(columns=["Assessment_Score"])

def predict_assessment(student_data):
    """
    Predict assessment score and decide if the student is promoted or demoted.
    """

    # Convert student_data into a DataFrame
    df = pd.DataFrame([student_data])

    # List of expected features (same as during training)
    expected_features = [
        "Age", "Student_Level", "Course_Level", "Study_Hours", "IQ_Score",
        "Gender", "Country", "State", "City", "Parent_Occupation", "Earning_Class",
        "Material_Level", "Course_Name", "Material_Name"
    ]

    # Ensure all missing features are added with default values
    for feature in expected_features:
        if feature not in df.columns:
            df[feature] = "Unknown"  # Assign default value for missing categorical columns

    # Encoding categorical values
    categorical_features = ["Student_Level", "Course_Level", "Material_Level",
                            "Course_Name", "Material_Name", "Gender", "Country", "State"]

    for col in categorical_features:
        df[col] = df[col].astype("category").cat.codes

    # Predict the assessment score
    predicted_score = model.predict(df)[0]

    # Promotion & Demotion Logic
    if predicted_score >= 60:
        decision = "Promoted to next level 🚀"
        student_data["Student_Level"] += 1
    elif predicted_score < 40:
        decision = "Demoted to previous level 🔻"
        student_data["Student_Level"] -= 1
    else:
        decision = "Stay in the current level 📍"

    return {
        "Predicted Score": round(predicted_score, 2),
        "Decision": decision,
        "Updated Student Level": student_data["Student_Level"]
    }

# Example student input
student_input = {
    "Age": 15,
    "Student_Level": 1,  # Intermediate
    "Course_Level": 1,  # Intermediate
    "Study_Hours": 3.5,
    "IQ_Score": 120,
    "Country": "India",
    "State": "TamilNadu",
    "City": "Chennai",
    "Parent_Occupation": "Engineer",
    "Earning_Class": "Middle",
    "Material_Level": 1,  # Intermediate Material
    "Course_Name": "Mathematics",
    "Material_Name": "Algebra"
}

# Run Prediction
result = predict_assessment(student_input)
print(result)


ValueError: The feature names should match those that were passed during fit.
Feature names must be in the same order as they were in fit.
